# How-to

This notebook serves as a practical guide to common questions users might have.

**Table of content**

- [Changing the parameters for an interaction](#Changing-the-parameters-for-an-interaction)
- [Writing your own interaction](#Writing-your-own-interaction)
- [Working with docking poses instead of MD simulations](#Working-with-docking-poses-instead-of-MD-simulations)
- [Using PDBQT files](#Using-PDBQT-files)



In [ ]:
import MDAnalysis as mda
import prolif as plf
import pandas as pd

In [ ]:
u = mda.Universe(plf.datafiles.TOP, plf.datafiles.TRAJ)
lig = u.select_atoms("resname LIG")
# restrict to 7.0 angstroms around the ligand
prot = u.select_atoms("protein and byres around 7 group ligand", ligand=lig)
lmol = plf.Molecule.from_mda(lig)
pmol = plf.Molecule.from_mda(prot)

## Changing the parameters for an interaction

You can list all the available interactions as follow:

In [ ]:
plf.Fingerprint.list_available(show_hidden=True)

In this example, we'll reparametrize the hydrophobic interaction with a shorter distance. To know which parameters are available for interactions, see the [prolif.interactions](../source/modules/interaction-fingerprint.html#detecting-interactions-between-residues-prolif-interactions-interactions) module.

Let's start with a test case: with the default parameters, Y109 is interacting with our ligand in 4 different occasions.

In [ ]:
fp = plf.Fingerprint()
hydrophobic_interactions = fp.hydrophobic(lmol["LIG1.G"], pmol["TYR109.A"])
len(list(hydrophobic_interactions))

Now we'll simply change the distance threshold to `4.0` instead of the default `4.5`.

In [ ]:
fp = plf.Fingerprint(parameters={"Hydrophobic": {"distance": 4.0}})
hydrophobic_interactions = fp.hydrophobic(lmol["LIG1.G"], pmol["TYR109.A"])
len(list(hydrophobic_interactions))

You can then use `fp.run` and other methods as usual.

### Repurposing an existing interaction

In case you want to reuse an existing class for a different type of interaction, the easiest way is to 
define an interaction class that inherits one of the classes listed in the `prolif.interactions` module, and just update its `__init__` method with the appropriate parameters:

In [ ]:
class ShorterHydrophobic(plf.interactions.Hydrophobic):
    def __init__(self, distance=4.0):
        super().__init__(distance=distance)


fp = plf.Fingerprint(["Hydrophobic", "ShorterHydrophobic"])
hydrophobic_interactions = fp.hydrophobic(lmol["LIG1.G"], pmol["TYR109.A"])
print(len(list(hydrophobic_interactions)))  # 4
shorterhydrophobic_interactions = fp.shorterhydrophobic(
    lmol["LIG1.G"], pmol["TYR109.A"]
)
print(len(list(shorterhydrophobic_interactions)))  # 0

## Writing your own interaction

Before you dive into this section, make sure that there isn't already an interaction that could just be **repurposed** to do what you want!

For this, the best is to check the section of the documentation corresponding to the `prolif.interactions` module. There are some generic interactions, like the `Distance` class, if you just need to define two chemical moieties within a certain distance. Both the Hydrophobic, Ionic, and Metallic interactions inherit from this class!
In most cases, defining an interaction only based on a distance is not enough and requires one or two angles constraints as well. For this purpose, the `SingleAngle` and `DoubleAngle` interactions can be used.

With that being said, there are a few rules that you must respect when writing your own interaction:

* **Inherit the ProLIF Interaction class**
  
This `prolif.interactions.Interaction` class in the base class that provides some functionalities to automatically register the interactions in `Fingerprint` objects.

* **Naming convention**

For non-symmetrical interactions, like hydrogen bonds or salt-bridges, the convention used here is to name the class after the role of the ligand. For example, the class `HBDonor` detects if a ligand acts as a hydrogen bond donor, and the class `Cationic` detects if a ligand acts as a cation.

* **Define a `detect` method**

This method takes exactly two positional arguments: a ligand Residue and a protein Residue (in this order).

* **Return value for the `detect` method**

You must yield a dictionary containing some basic metadata about the interaction when it is detected. To help with this process, the `metadata` method should be used (see example below) for which the arguments are listed here:

  * the input residues (`lig_res` and `prot_res` arguments, of type `rdkit.Chem.Mol`),
  * the indices of atoms responsible for the interaction, (`lig_indices` and `prot_indices` arguments, of type `tuple[int, ...]`),
  * any other relevant metric (distances or angles), named as you want. Distances should be in angstroms, and preferably named `distance`, and angles should be in degrees.
  

Note that you don't have to return anything if no interaction is detected for a pair of residues.

In [ ]:
import numpy as np
from scipy.spatial import distance_matrix


class CloseContact(plf.interactions.Interaction):
    def __init__(self, contact_threshold=2.0):
        self.contact_threshold = contact_threshold

    def detect(self, ligand_residue, protein_residue):
        # distance matrix between atoms of both residues
        dist_matrix = distance_matrix(ligand_residue.xyz, protein_residue.xyz)
        below_threshold = dist_matrix < self.contact_threshold
        for ligand_indices in np.argwhere(below_threshold.any(axis=1)):
            ligand_index = int(ligand_indices[0])
            for protein_indices in np.argwhere(below_threshold[ligand_index]):
                protein_index = int(protein_indices[0])
                # yield dict with metadata on the interaction
                # required arguments: input residues, and tuple of indices of atoms
                #                     responsible for the interaction
                # optional arguments: any additional `key=value` pair (e.g. distance)
                yield self.metadata(
                    lig_res=ligand_residue,
                    prot_res=protein_residue,
                    lig_indices=(ligand_index,),
                    prot_indices=(protein_index,),
                    distance=dist_matrix[ligand_index, protein_index],
                )


fp = plf.Fingerprint(["CloseContact"])
len(list(fp.closecontact(lmol["LIG1.G"], pmol["ASP129.A"])))

If you want to access metadata about the interaction, you can do one of the following:

* Recommended: use the `run` or `run_from_iterable` methods and then interact with the `fp.ifp` attribute:

In [ ]:
fp = plf.Fingerprint()
fp.run(u.trajectory[0:1], lig, prot)
# check the interactions between the ligand and ASP129 on the first frame
frame_number, ligand_residue, protein_residue = 0, "LIG1.G", "ASP129.A"
fp.ifp[frame_number][(ligand_residue, protein_residue)]

* For custom workflows: use the `metadata=True` argument when calling the `generate` method:

In [ ]:
ifp = fp.generate(lmol, pmol, metadata=True)
# check the interaction between the ligand and ASP129
ifp[("LIG1.G", "ASP129.A")]

* For debugging: use the `metadata=True` argument when calling the interaction directly through the fingerprint object:

In [ ]:
list(fp.closecontact(lmol["LIG1.G"], pmol["ASP129.A"], metadata=True))

* For debugging: use the `metadata` method instead of `bitvector`:

In [ ]:
fp.metadata(lmol["LIG1.G"], pmol["ASP129.A"])

## Working with docking poses instead of MD simulations

ProLIF currently provides file readers for MOL2, SDF and PDBQT files. The API is slightly different compared to the quickstart example but the end result is the same.

Please note that this part of the tutorial is only suitable for interactions between one protein and several ligands, or in more general terms, between one molecule with multiple residues and one molecule with a single residue. This is not suitable for protein-protein or DNA-protein interactions.

Let's start by loading the protein. Here I'm using a PDB file but you can use any format supported by MDAnalysis as long as it contains explicit hydrogens.

Note that for this tutorial, we're using example files that come with the package. These files are accessed through the `plf.datafiles.datapath` variable which holds a `pathlib.Path` object. This makes it easier to manipulate paths to file, match filenames using wildcards...etc. in a Pythonic way, but you can also use plain strings, i.e. `"/home/cedric/projects/ProLIF/prolif/data/vina/rec.pdb"` instead of `plf.datafiles.datapath / "vina" / "rec.pdb"` if you prefer.

In [ ]:
# load protein
prot = mda.Universe(plf.datafiles.datapath / "vina" / "rec.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

### Using an SDF file

In [ ]:
# load ligands
path = str(plf.datafiles.datapath / "vina" / "vina_output.sdf")
lig_suppl = plf.sdf_supplier(path)
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe()
df

If you want to calculate the Tanimoto similarity between your docked poses and a reference ligand, here's how to do it.

We first need to generate the interaction fingerprint for the reference, and concatenate it to the previous one

In [ ]:
# load the reference
ref = mda.Universe(plf.datafiles.datapath / "vina" / "lig.pdb")
ref = plf.Molecule.from_mda(ref)
# generate IFP
fp.run_from_iterable([ref], prot)
df0 = fp.to_dataframe()
df0.rename({0: "ref"}, inplace=True)
# drop the ligand level on both dataframes
df0.columns = df0.columns.droplevel(0)
df.columns = df.columns.droplevel(0)
# concatenate and sort columns
df = (
    pd.concat([df0, df])
    .fillna(False)
    .sort_index(
        axis=1, level=0, key=lambda index: [plf.ResidueId.from_string(x) for x in index]
    )
)
df

Lastly, we can convert the dataframe to a list of RDKit bitvectors to finally compute the Tanimoto similarity between our reference pose and the docking poses generated by Vina:

In [ ]:
from rdkit import DataStructs

bvs = plf.to_bitvectors(df)
for i, bv in enumerate(bvs[1:]):
    tc = DataStructs.TanimotoSimilarity(bvs[0], bv)
    print(f"{i}: {tc:.3f}")

Interestingly, the best scored docking pose (#0) isn't the most similar to the reference (#5)

### Using a MOL2 file

The input mol2 file can contain multiple ligands in different conformations.

In [ ]:
# load ligands
path = str(plf.datafiles.datapath / "vina" / "vina_output.mol2")
lig_suppl = plf.mol2_supplier(path)
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe()
df

If your protein is also a MOL2 file, here's a code snippet to guide you:

```python
u = mda.Universe("protein.mol2")
# add "elements" category
elements = mda.topology.guessers.guess_types(u.atoms.names)
u.add_TopologyAttr("elements", elements)
# create protein mol and run
prot = plf.Molecule.from_mda(u)
fp = plf.Fingerprint()
suppl = plf.mol2_supplier("ligands.mol2")
fp.run_from_iterable(suppl, prot)
df = fp.to_dataframe()
df
```


While doing so, you may run into these errors:

* **`RDKit ERROR: [17:50:37] Can't kekulize mol.  Unkekulized atoms: 1123`**
  This usually happens when some of the bonds in the MOL2 file are unconventional. For example, charged histidines are represented part with aromatic bonds and part with single and double bonds in MOE, presumably to capture the different charged resonance structures in a single one. A practical workaround for this is to redefine problematic bonds as single bonds in the Universe:

```python
u = mda.Universe("protein.mol2")
# replace aromatic bonds with single bonds
for i, bond_order in enumerate(u._topology.bonds.order):
    # you may need to replace double bonds ("2") as well
    if bond_order == "ar":
        u._topology.bonds.order[i] = 1
# clear the bond cache, just in case
u._topology.bonds._cache.pop("bd", None)
# infer bond orders again
prot = plf.Molecule.from_mda(u)
```


* **`RDKit ERROR: [17:50:37] non-ring atom 33 marked aromatic`**
  This is very similar to the previous error. You can use the same fix or, as an alternative, use RDKit:
  
```python
from rdkit import Chem
mol = Chem.MolFromMol2File("protein.mol2", removeHs=False)
# assign residue info (needed for fingerprint generation)
u = mda.Universe("protein.mol2")
for atom, resname in zip(mol.GetAtoms(), u.atoms.resnames):
    resid = plf.ResidueId.from_string(resname)
    mi = Chem.AtomPDBResidueInfo()
    mi.SetResidueNumber(resid.number)
    mi.SetResidueName(resid.name)
    atom.SetMonomerInfo(mi)
prot = plf.Molecule(mol)
```


* **Residue naming** in the resulting dataframe may be different from what was expected as the residue index is appended to the residue name and number:

```python
import numpy as np
u = mda.Universe("protein.mol2")
resids = [plf.ResidueId.from_string(x) for x in u.residues.resnames]
u.residues.resnames = np.array([x.name for x in resids], dtype=object)
u.residues.resids = np.array([x.number for x in resids], dtype=np.uint32)
u.residues.resnums = u.residues.resids
prot = plf.Molecule.from_mda(u)
```

### Using PDBQT files

The typical use case here is getting the IFP from AutoDock Vina's output. It requires a few additional steps and informations compared to other formats like MOL2, since the PDBQT format gets rid of most hydrogen atoms and doesn't contain bond order information.

The prerequisites for a successfull usage of ProLIF in this case is having external files that contain bond orders and formal charges for your ligand (like SMILES, SDF or MOL2), or at least a file with explicit hydrogen atoms. 

Please note that your PDBQT input must have a single model per file (this is required by MDAnalysis). Splitting a multi-model file can be done using the `vina_split` command-line tool that comes with AutoDock Vina: `vina_split --input vina_output.pdbqt`

Let's start by loading our "template" file with bond orders. It can be a SMILES string, MOL2, SDF file or anything supported by RDKit.

In [ ]:
from rdkit import Chem

template = Chem.MolFromSmiles(
    "C[NH+]1CC(C(=O)NC2(C)OC3(O)C4CCCN4C(=O)C(Cc4ccccc4)N3C2=O)C=C2c3cccc4[nH]cc(c34)CC21"
)
template

Next, we'll use the PDBQT supplier which loads each file from a list of paths, and assigns bond orders and charges using the template. The template and PDBQT file must have the exact same atoms, **even hydrogens**, otherwise no match will be found. Since PDBQT files partially keep the hydrogen atoms, we have the choice between:

* Manually selecting where to add the hydrogens on the template, do the matching, then add the remaining hydrogens (not covered here)
* Or just remove the hydrogens from the PDBQT file, do the matching, then re-add the original hydrogens.

For the protein, there's usually no need to load the PDBQT that was used by Vina. The original file that was used to generate the PDBQT can be used directly, but **it must contain explicit hydrogen atoms**:

In [ ]:
# load list of ligands
pdbqt_files = sorted(plf.datafiles.datapath.glob("vina/*.pdbqt"))
lig_suppl = plf.pdbqt_supplier(pdbqt_files, template)
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
df = fp.to_dataframe()
df